In [1]:
import numpy as np
from helper import *
from cleaning import clean_tweet
import nltk

Reading twitter - 1grams ...
Reading twitter - 2grams ...


C:\Users\ahmed\anaconda3\lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
print('Indexing word vectors.')

embeddings_index = {}
DIM_SIZE = 25
f = open('../Data/glove.twitter.27B.25d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 1193515 word vectors.


In [46]:
pos_train , neg_train = load_data()

In [47]:
len(pos_train.Text)

97889

In [48]:
clean_tweet("ah")

'ah'

In [49]:
def Tweet_to_GloVe(tweet):
    cleaned_tweet = clean_tweet(tweet)
    tweet_list = cleaned_tweet.split()
    list_of_embeddings = np.array([embeddings_index[word] for word in tweet_list],dtype='float32')
    return np.mean(list_of_embeddings, axis = 0)

In [50]:
Tweet_to_GloVe(pos_train.Text[0])

array([ 0.4451325 ,  0.5227325 ,  0.12862785, -0.0064234 , -0.271795  ,
       -0.5923875 ,  1.1854608 ,  0.61768526, -0.6002166 , -0.02863791,
       -0.35366833, -0.00649584, -3.921425  , -0.4466884 , -0.30765477,
        0.3058404 , -0.06779674, -0.51925206, -0.37925413, -0.36597148,
        0.16477333, -0.16300264, -0.28269824,  0.5131567 , -0.62351   ],
      dtype=float32)

In [51]:
x = np.zeros((len(pos_train.Text), DIM_SIZE))
i = 0
for  tweet in pos_train.Text:
    x[i] = Tweet_to_GloVe(tweet)
    i += 1

KeyError: 'ritika'

In [ ]:
DIM_SIZE = 25